## Modeling the musical difficulty

In [17]:
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [18]:
distributions = {
    "krumhansl_kessler": [
        0.15195022732711172, 0.0533620483369227, 0.08327351040918879,
        0.05575496530270399, 0.10480976310122037, 0.09787030390045463,
        0.06030150753768843, 0.1241923905240488, 0.05719071548217276,
        0.08758076094759511, 0.05479779851639147, 0.06891600861450106,

        0.14221523253201526, 0.06021118849696697, 0.07908335205571781,
        0.12087171422152324, 0.05841383958660975, 0.07930802066951245,
        0.05706582790384183, 0.1067175915524601, 0.08941810829027184,
        0.06043585711076162, 0.07503931700741405, 0.07121995057290496
    ],
    "aarden_essen": [
        0.17766092893562843, 0.001456239417504233, 0.1492649402940239,
        0.0016018593592562562, 0.19804892078043168, 0.11358695456521818,
        0.002912478835008466, 0.2206199117520353, 0.001456239417504233,
        0.08154936738025305, 0.002329979068008373, 0.049512180195127924,

        0.18264800547944018, 0.007376190221285707, 0.14049900421497014,
        0.16859900505797015, 0.0070249402107482066, 0.14436200433086013,
        0.0070249402107482066, 0.18616100558483017, 0.04566210136986304,
        0.019318600579558018, 0.07376190221285707, 0.017562300526869017
    ],
    "bellman_budge": [
        0.168, 0.0086, 0.1295, 0.0141, 0.1349, 0.1193,
        0.0125, 0.2028, 0.018000000000000002, 0.0804, 0.0062, 0.1057,

        0.1816, 0.0069, 0.12990000000000002,
        0.1334, 0.010700000000000001, 0.1115,
        0.0138, 0.2107, 0.07490000000000001,
        0.015300000000000001, 0.0092, 0.10210000000000001
    ],
    "temperley": [
        0.17616580310880825, 0.014130946773433817, 0.11493170042392838,
        0.019312293923692884, 0.15779557230334432, 0.10833725859632594,
        0.02260951483749411, 0.16839378238341965, 0.02449364107395195,
        0.08619877531794629, 0.013424399434762127, 0.09420631182289213,

        0.1702127659574468, 0.020081281377002155, 0.1133158020559407,
        0.14774085584508725, 0.011714080803251255, 0.10996892182644036,
        0.02510160172125269, 0.1785799665311977, 0.09658140090843893,
        0.016017212526894576, 0.03179536218025341, 0.07889074826679417
    ],
    'albrecht_shanahan1': [
        0.238, 0.006, 0.111, 0.006, 0.137, 0.094,
        0.016, 0.214, 0.009, 0.080, 0.008, 0.081,

         0.220, 0.006, 0.104, 0.123, 0.019, 0.103,
         0.012, 0.214, 0.062, 0.022, 0.061, 0.052
    ],
    'albrecht_shanahan2': [
        0.21169, 0.00892766, 0.120448, 0.0100265, 0.131444, 0.0911768, 0.0215947, 0.204703, 0.012894, 0.0900445, 0.012617, 0.0844338,

        0.201933, 0.009335, 0.107284, 0.124169, 0.0199224, 0.108324,
        0.014314, 0.202699, 0.0653907, 0.0252515, 0.071959, 0.049419
    ]    
}

In [19]:
def compute_threshold(dist_max, d, cutoff):    
    thresh = dist_max - (d * dist_max / cutoff)
    return thresh

def clipped_distribution(orig_dist, d, cutoff):
    # make a copy of the original distribution
    copy = np.array(orig_dist)
    # compute the threshold to get rid of difficult notes at initial difficulties
    threshold = compute_threshold(max(copy), d, cutoff)
    # remove the most difficult notes for low difficulties
    copy[copy < threshold] = 0.0
    # norm-1 of the distribution
    copy = copy / sum(copy)
    return copy, threshold

In [20]:
def scaled_distribution(clipped_dist, h, d):
    # make a copy of the original distribution
    copy = np.array(clipped_dist)        
    # compute the scaling factor based on handicap parameter and difficulty (user input)
    scaling = h - (h * d)
    # scale the distribution
    copy = copy ** scaling
    # norm-1 of the distribution
    copy = copy / sum(copy)
    return copy

In [23]:
def f(dist_name, clipping, handicap, difficulty):
    # create the figures
    f, (axmaj, axmin) = plt.subplots(2, 3, sharex=True, sharey=True)
    
    # get the original distributions for major and minor keys
    dist = np.array(distributions[dist_name])
    major = dist[:12]
    minor = dist[12:]
    
    # clip the distributions for lower difficulties
    clipped_major, major_threshold = clipped_distribution(major, difficulty, clipping)
    clipped_minor, minor_threshold = clipped_distribution(minor, difficulty, clipping)
    
    # get the scaled distribution according to difficulty, handicap, and initial clipping    
    scaled_major = scaled_distribution(clipped_major, handicap, difficulty)
    scaled_minor = scaled_distribution(clipped_minor, handicap, difficulty)    
    
    # prepare to plot
    x = np.array(['C', 'C#', 'D', 'Eb', 'E', 'F',
                  'F#', 'G', 'Ab', 'A', 'Bb', 'B'])    
    
    sns.barplot(x=x, y=major, ax=axmaj[0])
    axmaj[0].set_ylabel("Original Major")
    axmaj[0].axhline(major_threshold, color="k", clip_on=True)
    
    sns.barplot(x=x, y=clipped_major, ax=axmaj[1])
    axmaj[1].set_ylabel("Clipped Major")
    
    sns.barplot(x=x, y=scaled_major, ax=axmaj[2])
    axmaj[2].set_ylabel("Scaled Major")
    
    sns.barplot(x=x, y=minor, ax=axmin[0])
    axmin[0].set_ylabel("Original Minor")
    axmin[0].axhline(minor_threshold, color="k", clip_on=True)
    
    sns.barplot(x=x, y=clipped_minor, ax=axmin[1])
    axmin[1].set_ylabel("Clipped Minor")
    
    sns.barplot(x=x, y=scaled_minor, ax=axmin[2])
    axmin[2].set_ylabel("Scaled Minor")
    
    plt.tight_layout(h_pad=2)
    

In [24]:
distribution_name = list(distributions.keys())
handicap = widgets.IntSlider(min=1, max=10, continuous_update=False)
difficulty = widgets.FloatSlider(min=0.0, max=1.0, step=0.01, continuous_update=False)
clipping = widgets.FloatSlider(min=0.2, max=0.8, step=0.1, continuous_update=False)
interact(f, dist_name=distribution_name, handicap=handicap, difficulty=difficulty, clipping=clipping)

interactive(children=(Dropdown(description='dist_name', options=('krumhansl_kessler', 'aarden_essen', 'bellman…

<function __main__.f(dist_name, clipping, handicap, difficulty)>